<a href="https://colab.research.google.com/github/andrewfeikema/alpha-zero-general/blob/master/Othello_Train_Trial_using_AlphaZero_General.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup, install dependencies**

In [1]:
# Clone repo and install requirements

!git clone https://github.com/andrewfeikema/alpha-zero-general.git

Cloning into 'alpha-zero-general'...
remote: Enumerating objects: 1160, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 1160 (delta 47), reused 95 (delta 39), pack-reused 1051
Receiving objects: 100% (1160/1160), 992.12 MiB | 26.17 MiB/s, done.
Resolving deltas: 100% (623/623), done.
Checking out files: 100% (62/62), done.


In [2]:
%cd '/content/alpha-zero-general'

/content/alpha-zero-general


In [3]:
!git checkout -t origin/master

fatal: A branch named 'master' already exists.


In [4]:
!pip install -r docker/requirements.txt

     |████████████████████████████████| 51kB 3.1MB/s 
     |████████████████████████████████| 2.0MB 5.8MB/s 
     |████████████████████████████████| 450kB 50.3MB/s 
     |████████████████████████████████| 31.2MB 165kB/s 
     |████████████████████████████████| 9.5MB 54.7MB/s 
     |████████████████████████████████| 25.3MB 1.5MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 92kB 12.4MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
  ERROR: Failed building wheel for scikit-learn
  Running setup.py clean for scikit-learn
  Created wheel for torchfile: filename=torchfile-0.1.0-cp37-none-any.whl size=5713 sha256=5dd559c530f73b9f20d6f33dfb9db0be9ee51409e3d12ce23c32fcbcab2972f1
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
  Created wheel for visdom: filename=visdom

# **Train the AlphaZero model**

In [5]:
import logging
import coloredlogs
from Coach import Coach
from utils import dotdict
from othello.pytorch.NNet import NNetWrapper
from othello.OthelloGame import OthelloGame

In [6]:
log = logging.getLogger(__name__)
coloredlogs.install(level='INFO')  # Change this to DEBUG to see more info.

In [7]:
args = dotdict({
    'numIters': 1000,
    'numEps': 100,              # Number of complete self-play games to simulate during a new iteration.
    'tempThreshold': 15,        #
    'updateThreshold': 0.6,     # During arena playoff, new checkpoint model will be accepted if threshold or more of games are won.
    'maxlenOfQueue': 200000,    # Number of game examples to train the neural networks.
    'numMCTSSims': 25,          # Number of games moves for MCTS to simulate.
    'arenaCompare': 40,         # Number of games to play during arena play to determine if new net will be accepted.
    'cpuct': 1,
    'checkpoint': './temp/',
    'load_model': False,
    'numItersForTrainExamplesHistory': 20,
})

In [8]:
# Load best recent model (2nd generation at '/5-pm-checkpoints')
import os
if os.path.exists(os.path.join('/pretrained_models/othello/keras/5-pm-checkpoints/', 'best.pth.tar')):
  print ("Using best pre-existing model")
  args['load_model'] = True
  args['load_folder_file'] = ('/pretrained_models/othello/keras/5-pm-checkpoints/','best.pth.tar')
else:
  print ("Not using best pre-existing model")

Not using best pre-existing model


In [9]:

# Set iterations and Epochs set to run about 1 hour
args['numIters'] = 10
args['numEps'] = 10
args['arenaCompare'] = 4

In [10]:
game = OthelloGame(n=8)

In [11]:
nnet = NNetWrapper(game)

In [12]:
if args.load_model:
    print('Loading checkpoint "{}/{}"...'.format(args.load_folder_file[0], args.load_folder_file[1]))
    nnet.load_checkpoint(args.load_folder_file[0], args.load_folder_file[1])
else:
    print('Not loading a checkpoint.')

Not loading a checkpoint.


In [13]:
coach = Coach(game, nnet, args)

In [31]:
%time coach.learn()

2021-05-12 20:22:41 bb450b7c5a31 Coach[71] INFO Starting Iter #1 ...
Training Net:   0%|          | 0/846 [00:00<?, ?it/s, Loss_pi=5.97e-01, Loss_v=2.89e-01]

Checkpoint Directory exists! 
EPOCH ::: 1


Training Net:   1%|          | 7/846 [00:00<00:12, 68.37it/s, Loss_pi=5.37e-01, Loss_v=2.01e-01]

EPOCH ::: 2


Training Net:   1%|          | 7/846 [00:00<00:12, 68.30it/s, Loss_pi=5.08e-01, Loss_v=1.75e-01]

...

EPOCH ::: 9


Training Net:   0%|          | 7/1503 [00:00<00:21, 69.14it/s, Loss_pi=4.17e-01, Loss_v=1.69e-01]

EPOCH ::: 10


Training Net: 100%|██████████| 1503/1503 [00:22<00:00, 65.48it/s, Loss_pi=3.76e-01, Loss_v=1.55e-01]
2021-05-12 21:02:50 bb450b7c5a31 Coach[71] INFO PITTING AGAINST PREVIOUS VERSION
Arena.playGames (2): 100%|██████████| 2/2 [00:06<00:00,  3.38s/it]
2021-05-12 21:03:03 bb450b7c5a31 Coach[71] INFO NEW/PREV WINS : 3 / 1 ; DRAWS : 0
2021-05-12 21:03:03 bb450b7c5a31 Coach[71] INFO ACCEPTING NEW MODEL


Checkpoint Directory exists! 
Checkpoint Directory exists! 
CPU times: user 40min 19s, sys: 1min 2s, total: 41min 21s
Wall time: 40min 22s
